# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
!pip install python-dotenv


In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-4o",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [6]:
#Definition of the tables.
import pandas as pd

# Define table metadata
data = [
    {'table': 'employees', 'definition': 'Employee information, name, department, salary'},
    {'table': 'salary', 'definition': 'Salary details for each year'},
    {'table': 'studies', 'definition': 'Educational studies, name of the institution, type of studies, level'}
]

df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees     Employee information, name, department, salary
1     salary                       Salary details for each year
2    studies  Educational studies, name of the institution, ...


In [7]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [8]:
print(text_tables)

employees: Employee information, name, department, salary
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level


In [48]:
prompt_question_tables = """Given the following tables and their content definitions:

### Tables
{tables}

### SQL Table Definitions:
```sql
CREATE TABLE customer_orders (
    order_id INTEGER PRIMARY KEY,
    customer_name VARCHAR(50),
    product_name VARCHAR(100),
    order_date DATE,
    quantity INTEGER,
    total_price DECIMAL(10,2)
);

CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name VARCHAR(50),
    email VARCHAR(100),
    city VARCHAR(50),
    country VARCHAR(50)
);

CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name VARCHAR(100),
    category VARCHAR(50),
    price DECIMAL(10,2),
    stock_quantity INTEGER
);

### Response Format:
You MUST return ONLY a JSON object. Do NOT include explanations or any extra text.

The response MUST be in the following format:

```json
{{
  "tables": ["table_1", "table_2"]
}}
"""


In [49]:
pqt1 = prompt_question_tables.format(tables=formatted_tables, question="Find the average salary of employees in each department.")

In [50]:
print(return_OAI(pqt1))

```json
{
  "tables": ["customer_orders", "customers", "products"]
}
```


In [51]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Find employees who have a degree in Computer Science and their salary.")

In [52]:
print(return_OAI(pqt3))

```json
{
  "tables": ["customer_orders", "customers", "products"]
}
```


# My reflections:

I see that through giving it examples of tables and in the ### reponse explicity tell it to output the answer in Json. At first I did not explicity tell it to output JSON and it failed to answer correctly.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?